In [1]:
!pip install -q -U openai-whisper
!pip install -q -U edge-tts
!pip install -q -U webrtcvad
!pip install -q -U wave
!pip install -q -U pydub
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -U bitsandbytes
!pip install -q -U librosa
!pip install -q -U huggingface_hub
!pip install -q -U gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 8.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
import librosa

def all_zeros(lst):
    return all(x == 0 for x in lst)



def energy_based_vad(audio_signal, sample_rate, threshold=0.5, frame_duration=0.02):

    frame_length = int(frame_duration * sample_rate)
    num_frames = int(len(audio_signal) / frame_length)

    # Compute energy of each frame
    frame_energy = np.array([np.sum(np.square(audio_signal[i*frame_length:(i+1)*frame_length]))
                             for i in range(num_frames)])

    # Normalize energy
    max_energy = np.max(frame_energy)
    if max_energy > 0:
        frame_energy = frame_energy / max_energy

    # Apply threshold
    vad_result = (frame_energy > threshold).astype(int)

    return all_zeros(vad_result)

# Example usage
# Load an audio file (you can replace this with any audio file)
audio_path = "query2.wav"
audio_signal, sample_rate = librosa.load(audio_path, sr=None)

# Apply VAD
vad_result = energy_based_vad(audio_signal, sample_rate, threshold=0.5)


In [ ]:
import re

def parse_response(response):
    """
    Parses the response to extract the part after '###Response' and before the next '#'.

    Parameters:
    - response (str): The response string to parse.

    Returns:
    - str: The part of the response after '###Response' and before the next '#'.
    """
    # Define the regex pattern to match '###Response' followed by content until the next '#'
    pattern = r'###Response\s*(.*?)(?=\s*#|$)'

    # Use re.DOTALL to ensure '.' matches newline characters as well
    match = re.search(pattern, response, re.DOTALL)

    # If a match is found, return the captured content
    if match:
        return match.group(1).strip()

    # If no match is found, return an empty string
    return ''

In [ ]:
import whisper
from transformers import pipeline
import edge_tts
import asyncio
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import os


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


# Step 1: Voice-to-Text Conversion
def voice_to_text(audio_file_path):
    model_whisper = whisper.load_model("base.en")
    audio_signal, sample_rate = librosa.load(audio_path, sr=None)
    vad_result = energy_based_vad(audio_signal, sample_rate, threshold=0.5)
    if vad_result == False:
        result = model_whisper.transcribe(audio_file_path)
        return result['text']
    else:
      print("No text detected")
      return None

# Step 2: Text Input into LLM and Response Generation
def generate_response(text_input):
    prompt = f"### Instruction: Answer the following query {text_input}\n### Response:"
    sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
    )

    return sequences[0]['generated_text']

def text_to_speech(text, lang = 'en', pitch_change=-1, rate_change=1.0, output_file="modified_output.wav"):

    tts = gTTS(text=text, lang=lang)
    tts.save("temp_output.mp3")

    audio = AudioSegment.from_mp3("temp_output.mp3")
    audio.export("temp_output.wav", format="wav")

    sound = AudioSegment.from_wav("temp_output.wav")

    def change_pitch(sound, octaves):
        new_sample_rate = int(sound.frame_rate * (2 ** octaves))
        return sound._spawn(sound.raw_data, overrides={'frame_rate': new_sample_rate}).set_frame_rate(sound.frame_rate)

    def change_rate(sound, rate):
        return sound._spawn(sound.raw_data, overrides={'frame_rate': int(sound.frame_rate * rate)}).set_frame_rate(sound.frame_rate)

    modified_sound = change_pitch(sound, pitch_change)
    modified_sound = change_rate(modified_sound, rate_change)

    modified_sound.export(output_file, format="wav")

    os.remove("temp_output.mp3")
    os.remove("temp_output.wav")

    print(f"Modified audio saved to {output_file}")


def run_pipeline(audio_file, pitch_change, rate_change):

    text = voice_to_text(audio_file)
    print("Transcribed Text:", text)

    llm_response = generate_response(text)
    print("LLM Response:", llm_response)
    response = parse_response(llm_response)
    print("Parsed Response:", response)

    text_to_speech(response, lang='en', pitch_change = pitch_change, rate_change = rate_change, output_file='output.mp3')
    return llm_response

audio_file = "query2.wav"
llm_response = run_pipeline(audio_file, -0.5, 2.0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]